In [1]:
import torch
import torchvision
from torchvision import transforms, datasets

In [2]:
Tl_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.0229, 0.224, 0.225])
])

In [3]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights.meta , weights

weights.meta.keys()

dict_keys(['categories', 'min_size', 'recipe', 'num_params', '_metrics', '_docs'])

In [4]:
len(weights.meta['categories']) #* imagenet classes, so impressive


1000

In [5]:
weights.meta['recipe'] #* recipe for the model architecture

'https://github.com/pytorch/vision/tree/main/references/classification#efficientnet-v1'

In [6]:
weights.meta['num_params']

5288548

In [7]:
weights.meta['_metrics']

{'ImageNet-1K': {'acc@1': 77.692, 'acc@5': 93.532}}

In [8]:
weights.meta['_docs']

'These weights are ported from the original paper.'

In [9]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [10]:
train_dataset = datasets.ImageFolder(
    root='data/images/train/',
    transform=auto_transforms,
    target_transform=None, 
)

In [11]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=16, 
    shuffle=True,
)

In [12]:
#* tl model

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
tl_model = torchvision.models.efficientnet_b0(weights=weights).to('cuda')

In [13]:
# import torchinfo

# torchinfo.summary(
#     model = tl_model,
#     input_size = (1, 3, 224, 224),
#     col_names = ["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
# )

In [14]:
tl_model.__dict__['_modules'].keys()

odict_keys(['features', 'avgpool', 'classifier'])

In [15]:
from torch import nn
tl_model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=3, bias=True)
tl_model.classifier

tl_model = tl_model.to('cuda')

In [48]:

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(
    params=tl_model.parameters(),
    lr = 0.0005
)

In [49]:
len(train_dataloader)

15

In [50]:
epochs = 5
counter = 0
total_loss = 0
loss_hist = []

for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (x, y) in enumerate(train_dataloader):
        x, y = x.type(torch.FloatTensor).to('cuda'), y.to('cuda')
        tl_model.train()
        logits = torch.softmax(tl_model(x), dim=1)
        loss = loss_fn(logits, y)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()
        
        #torch.cuda.empty_cache()
    loss_hist.append(total_loss/len(train_dataloader))

    



In [42]:
x.dtype, y.dtype, logits.dtype

(torch.float32, torch.int64, torch.float32)

In [51]:
loss_hist

[1.0820165793100993,
 0.9997780124346415,
 1.0625171065330505,
 1.0815341035525003,
 1.0650770584742228]

In [22]:
torch.cuda.empty_cache()

In [23]:
torch.cuda.empty_cache()